In [1]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\doosa\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\doosa\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this syst

In [10]:
# 양자화 셋팅
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

input_variables=['product'] template='{product}를 홍보하기 위한 좋은 문구를 추천해줘'


In [12]:
# 모델과 토크나이저 호출
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

TypeError: ForwardRef._evaluate() missing 1 required keyword-only argument: 'recursive_guard'

In [ ]:
# call_llm에서 사용했던 부분을 축소해서 적어둔 것 (출력 형식이 정리되어있지 않음)
device = "cuda:0"

messages = [
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):  # pip install시 utf-8, ansi 관련 오류에 필요한 코드
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu langchain_community # 랭체인 등 Rag에 필요한 패키지 설치

In [ ]:
from langchain.llms import HuggingFacePipeline  # API 사용하면 필요없음 (openai 등)
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain  # 이 부분도 오픈소스 사용할 때 가져오는 부분

text_generation_pipeline = pipeline(  # 우리의 파이프라인 선언 토크나이저랑 모델 등 전부 담기
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,  # 일관된 답변 0, 다양한 답변일수록 1에 가까움
    return_full_text=True,  # 전체 텍스트를 받게 함
    max_new_tokens=300,  # 새로운 토큰 300개 설정
)


# 아래 템플릿 설명
# your {} knowledge에 원하는 정보를 넣으면 관련 정보를 가져옴
# context는 근거에 대한 문장 내용
# question에 질문이 담김

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=llm, prompt=prompt)